<a href="https://colab.research.google.com/github/chrishg23-jpg/HES-benchmark/blob/main/HES_3D_Ane2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

# Parameters
L = 10
steps = 20
alpha = 0.4      # stronger local interaction
beta = 0.005     # lighter damping
gamma = 0.0001   # gentler global constraint
threshold = 0.002  # curvature sensitivity

# Initialize lattice
s = np.random.uniform(-1.0, 1.0, (L, L, L))

# Seed asymmetry
s[5:10, 5:10, 5:10] += 0.5  # expansion pocket
s[12:15, 12:15, 12:15] -= 0.5  # contraction pocket

# Laplacian function
def laplacian_3d(s):
    return (
        np.roll(s, 1, axis=0) + np.roll(s, -1, axis=0) +
        np.roll(s, 1, axis=1) + np.roll(s, -1, axis=1) +
        np.roll(s, 1, axis=2) + np.roll(s, -1, axis=2) - 6 * s
    )

# Prepare figure
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111, projection='3d')
ax.set_xlim(0, L)
ax.set_ylim(0, L)
ax.set_zlim(0, L)
ax.set_title("Emergent Curvature in 3D HES Lattice")

# Animation update function
def update(frame):
    global s
    lap = laplacian_3d(s)
    global_entropy = np.mean(s)
    s += alpha * lap - beta * s - gamma * global_entropy
   # s += np.random.normal(0, 0.01, s.shape)  # inject noise

    # Expansion zones (positive curvature)
    expanding = lap > threshold
    x1, y1, z1 = np.where(expanding)
    intensity1 = lap[expanding]

    # Contraction zones (negative curvature)
    contracting = lap < -threshold
    x2, y2, z2 = np.where(contracting)
    intensity2 = -lap[contracting]  # flip sign for size scaling

    ax.clear()
    ax.set_xlim(0, L)
    ax.set_ylim(0, L)
    ax.set_zlim(0, L)
    ax.set_title(f"Step {frame}: Expansion vs Contraction")

    # Plot expansion in red
    ax.scatter(x1, y1, z1, c='red', s=10 * intensity1)

    # Plot contraction in blue
    ax.scatter(x2, y2, z2, c='blue', s=10 * intensity2)

# Run animation
anim = FuncAnimation(fig, update, frames=steps, interval=200)
plt.close()  # prevent duplicate static plot
HTML(anim.to_jshtml())
